# Predicciones con modelo Kneser–Ney

## 1) Cargamos el modelo

In [3]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../../..'))
from anexos.models.kn_gs_v2.kn_model import KNInterpolatedNGram 
import joblib
best_kn = joblib.load('../../models/kn_gs_v2/best_kn_model.pkl') # ajustar ruta si fuera necesario

# 2) Definimos la función de predicción

In [4]:
def topk_next(model, context, k=5, exclude_special=True):
    """Devuelve las k mejores sugerencias (token, prob) dado el contexto."""
    ranking = model.predict_ranking(context)
    if exclude_special:
        ranking = [(w,p) for (w,p) in ranking if w not in {"<s>", "</s>", "<unk>"}]
    return ranking[:k]

## 3) Evaluamos el rendimiento del modelo Kneser–Ney con algunas progresiones clásicas de acordes

### Modo mayor

In [5]:
#@title Echamos un vistazo a las sugerencias desde el acorde de tónica.
pred = topk_next(best_kn, ["I"], k=5)
pred

[('I', 0.3734150220734017),
 ('ii', 0.10753082407664535),
 ('vi', 0.105420704083276),
 ('V7', 0.052316504322354795),
 ('IV', 0.04821778901796282)]

In [4]:
#@title ["II", "V7"] > "I" 
pred = topk_next(best_kn, ["II", "V7"], k=5) # top1
pred

[('I', 0.5147374520306233),
 ('V7', 0.20810002151938234),
 ('I7', 0.06348083569710526),
 ('II', 0.05433766157067823),
 ('i', 0.029277076949103812)]

In [5]:
#@title pop-punk ["I", "V"] >  "vi" "IV" 
pred = topk_next(best_kn, ["I", "V"], k=5) # top4 : vi
pred

[('V', 0.3619163310072302),
 ('IV', 0.3145394182731477),
 ('I', 0.12915102984178167),
 ('vi', 0.09827329856710684),
 ('v', 0.04347285267236621)]

In [ ]:
#@title pop-punk ["I", "V", "vi"] > "IV" 
pred = topk_next(best_kn, ["I", "V", 'vi'], k=10) # top7 : IV
pred

[('II7', 0.21186953407468012),
 ('V', 0.16701901045989845),
 ('V/V', 0.1431200511922982),
 ('Vsub/II', 0.1044223682400282),
 ('vi', 0.09645031890317396),
 ('ii', 0.09193630138192463),
 ('IV', 0.07803668716106835),
 ('II', 0.05532994827095102),
 ('iii', 0.011144622733064845),
 ('vii', 0.00932151579384689)]

In [ ]:
#@title ["I", "IV"] > "I" 
pred = topk_next(best_kn, ["I", "IV"], k=5) # top1 : I
pred

[('I', 0.5523479680300832),
 ('iii', 0.17868779919129707),
 ('V7', 0.09357061851630752),
 ('V', 0.07075552203473516),
 ('IV', 0.06705918922323578)]

In [ ]:
#@title ["IV", "iv"] > "I" 
pred = topk_next(best_kn, ["IV", "iv"], k=5) # top1
pred

[('I', 0.6852342845268492),
 ('bVII7', 0.0911883723274025),
 ('iii', 0.07167197964826987),
 ('iv', 0.05531088804241253),
 ('V', 0.0168095248648733)]

In [ ]:
#@title ["i", "ii", "iii"] > "ii" o "IV"
pred = topk_next(best_kn, ["I", "ii", "iii"], k=5) # top1 top2
pred

[('ii', 0.2536295068957499),
 ('IV', 0.19301160791927907),
 ('vi', 0.12225064852545692),
 ('V/II', 0.11544736712344839),
 ('biiio', 0.0866727151744685)]

In [11]:
#@title ["vi", "ii"] > "V7"
pred = topk_next(best_kn, ["vi", "ii"], k=5) # top1
pred

[('V7', 0.8007619646811053),
 ('ii', 0.0912653006604839),
 ('vi', 0.030674614403312553),
 ('Vsub/III', 0.020604351350864265),
 ('I', 0.010689323093255666)]

In [12]:
#@title ["I", "bVII"] > "I"
pred = topk_next(best_kn, ["I", "bVII"], k=5) # top1
pred

[('I', 0.5923288712978145),
 ('IV', 0.2997835833604025),
 ('bVII', 0.038251036525888314),
 ('V/IV', 0.01421183877548663),
 ('V7', 0.013465340638542427)]

### Modo menor

In [ ]:
pred = topk_next(best_kn, ["i"], k=5)
pred

[('i', 0.4567366640398732),
 ('iiø', 0.11593073164806672),
 ('V7', 0.09815538622614726),
 ('VI', 0.049291912898090054),
 ('iv', 0.043694122784600234)]

In [13]:
#@title ["iiø", "V7"] > "i"
pred = topk_next(best_kn, ["iiø", "V7"], k=5) # top1
pred

[('i', 0.3797665999357742),
 ('I', 0.3726490852969921),
 ('V7', 0.10379978030365061),
 ('iiø', 0.092943737965714),
 ('VI', 0.005684761530808764)]

In [14]:
#@title ["VI", "iiø"] > "V7"
pred = topk_next(best_kn, ["VI", "iiø"], k=5) # top1
pred

[('V7', 0.9499133790613948),
 ('iiø', 0.0404212015707477),
 ('i', 0.0018882724017321784),
 ('I', 0.0014848214171321683),
 ('bVI7', 0.0013082564348126887)]

In [15]:
#@title ["i", "iv"] > "V7"
pred = topk_next(best_kn, ["i", "iv"], k=5) # top1
pred

[('V7', 0.3352135194999175),
 ('i', 0.2714959128569826),
 ('iv', 0.15174976164566029),
 ('V/III', 0.07723979242357881),
 ('bVII7', 0.04196969293049771)]

### Probamos con dominantes secundarios o sustitutos

In [37]:
#@title ["i", "V/IV"] > "iv"
pred = topk_next(best_kn, ["i","V/IV"], k=5) # top1
pred

[('iv', 0.8902333980552132),
 ('IV7', 0.057264088830162325),
 ('IV', 0.051865587765010976),
 ('Vsub/III', 0.0003623594832244842),
 ('ivo', 0.00011519815686804315)]

In [21]:
#@title ["I", "Vsub/V"] > "V7"
pred = topk_next(best_kn, ["I", "Vsub/V"], k=5) # top1
pred

[('V7', 0.9681584477001454),
 ('v', 0.0282699915704652),
 ('V', 0.003155644318534363),
 ('vø', 0.00010456807308590371),
 ('vi', 6.943650702937479e-06)]

In [27]:
#@title ["I", "biio"] > "ii"
pred = topk_next(best_kn, ["I", "biio"], k=5) # top1
pred

[('ii', 0.8636335920722978),
 ('V7', 0.12125336491141733),
 ('v', 0.006953592240068002),
 ('V/V', 0.0014713261117573371),
 ('biio', 0.001166692577845383)]

## Componiendo una progresión

In [4]:
pred = topk_next(best_kn, ["i", 'iiø'], k=5)
pred

[('V7', 0.9886360585969368),
 ('i', 0.009514461811495038),
 ('iiø', 0.0017383814376694122),
 ('I', 3.0499034514066163e-05),
 ('ii', 1.239012868759704e-05)]

In [5]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i"], k=5)
pred

[('i', 0.3797404604722146),
 ('iv', 0.08423017960051365),
 ('V7', 0.072119240695104),
 ('IV7', 0.049583448385923795),
 ('VI', 0.04470348488940704)]

In [6]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv"], k=5)
pred

[('iv', 0.46080449778367755),
 ('V/III', 0.18703886757562485),
 ('V7', 0.12931543117271077),
 ('i', 0.09611736438957834),
 ('bVII7', 0.032257574949735905)]

In [8]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv", 'V/III'], k=5)
pred

[('III', 0.9939206301405261),
 ('iii', 0.005478190871580312),
 ('iiio', 0.00042881693117342943),
 ('V/VI', 8.180946641489668e-05),
 ('III7', 4.003178716872721e-05)]

In [9]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv", 'V/III', 'III'], k=5)
pred

[('III', 0.27511384963495844),
 ('VI', 0.22096722613743172),
 ('iiø', 0.0757880053672258),
 ('i', 0.050073249851847815),
 ('V7', 0.046598375713362845)]

In [10]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv", 'V/III', 'III', 'III'], k=5)
pred

[('III', 0.2639272806588385),
 ('iiø', 0.15022034452402674),
 ('iii', 0.08962676098542652),
 ('V7', 0.07574027016341532),
 ('ii', 0.05883997728380215)]

In [11]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv", 'V/III', 'III', 'III', 'iiø'], k=5)
pred

[('V7', 0.7434092941769465),
 ('iiø', 0.20041885873224874),
 ('bII7', 0.052511302977086206),
 ('vi', 0.0024263187366431044),
 ('I', 0.0002747883791445896)]

In [12]:
pred = topk_next(best_kn, ["i", 'iiø', "i", "i", "iv", 'V/III', 'III', 'III', 'iiø', 'V7'], k=5)
pred

[('i', 0.6748276022252877),
 ('I', 0.04457034709436917),
 ('natvi', 0.02527255812939513),
 ('V7', 0.016443019542469205),
 ('iiø', 0.008424351906493283)]